In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re
from pyvi.ViTokenizer import tokenize

In [117]:
df1= pd.read_csv('Data_Agoda_raw/hotel_info.csv')
df1.head()

,num,Hotel_ID,Hotel_Name,Hotel_Rank,Hotel_Address,Total_Score,Location,Cleanliness,Service,Facilities,Value_for_money,Comfort_and_room_quality,comments_count,Hotel_Description
0,1,1_1,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,5 sao trên 5,"60 Trần Phú, Lộc Thọ, Nha Trang, Việt Nam","8,8","9,4","8,9","8,9","8,7","8,7","8,3",1269,Khách sạn Mường Thanh Luxury Nha Trang - Nơi l...
1,2,1_2,ALPHA BIRD NHA TRANG,4 sao trên 5,"51/19/37 Tue Tinh St, Loc Tho Ward, Nha Trang,...","7,7","7,8","7,6","8,1","7,5","8,1",NaN,337,ALPHA BIRD NHA TRANG - Khách sạn 4.0 sao tại N...
2,3,1_3,Khách sạn Aaron (Aaron Hotel),3.5 sao trên 5,"6Trần Quang Khải, Lộc Thọ, Nha Trang, Việt Nam...","8,5","8,9","8,7","8,8","8,1","8,5",NaN,300,Khách sạn Aaron - Nơi nghỉ dưỡng tuyệt vời tại...
3,4,1_4,Panorama Star Beach Nha Trang,5 sao trên 5,"02 Nguyen Thi Minh Khai, Lộc Thọ, Nha Trang, V...","8,8","9,6","8,9","8,9","8,7","9,0",NaN,814,Panorama Star Beach Nha Trang - Một kỳ nghỉ tu...
4,5,1_5,Khách sạn Balcony Nha Trang (Balcony Nha Trang...,4 sao trên 5,"98B/13 Trần Phú, Lộc Thọ, Nha Trang, Việt Nam","8,4","8,5","8,7","8,5","8,3","8,6","8,7",294,Khách sạn Balcony Nha Trang - Nơi nghỉ dưỡng t...


In [118]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   num                       740 non-null    int64 
 1   Hotel_ID                  740 non-null    object
 2   Hotel_Name                740 non-null    object
 3   Hotel_Rank                740 non-null    object
 4   Hotel_Address             740 non-null    object
 5   Total_Score               740 non-null    object
 6   Location                  413 non-null    object
 7   Cleanliness               412 non-null    object
 8   Service                   373 non-null    object
 9   Facilities                370 non-null    object
 10  Value_for_money           410 non-null    object
 11  Comfort_and_room_quality  51 non-null     object
 12  comments_count            740 non-null    int64 
 13  Hotel_Description         739 non-null    object
dtypes: int64(2), object(12)
me

In [119]:
num_cols = ["Total_Score", "Location", "Cleanliness", "Service","Facilities", "Value_for_money", "Comfort_and_room_quality", "comments_count"]

In [120]:
dirty_vals = ["no information", "nan", "none", "null", "", "na"]

In [121]:
# Chọn các cột object
obj_cols = df1.select_dtypes(include=["object"]).columns

summary = {}
for col in obj_cols:
    # Lấy tất cả giá trị duy nhất trong cột (chuẩn hóa về chữ thường)
    unique_vals = df1[col].astype(str).str.strip().str.lower().unique()
    
    # Kiểm tra giao giữa unique_vals và dirty_vals
    dirty_found = set(unique_vals) & set(dirty_vals)
    
    if dirty_found:
        summary[col] = list(dirty_found)
summary

{'Hotel_Rank': ['no information'],
 'Total_Score': ['no information'],
 'Location': ['nan'],
 'Cleanliness': ['nan'],
 'Service': ['nan'],
 'Facilities': ['nan'],
 'Value_for_money': ['nan'],
 'Comfort_and_room_quality': ['nan'],
 'Hotel_Description': ['nan']}

In [122]:
for col in num_cols:
    df1[col] = (
        df1[col]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace(dirty_vals, np.nan)
        .str.replace(",", ".", regex=False)
    )
    df1[col] = pd.to_numeric(df1[col], errors="coerce")

In [123]:
df1['Hotel_Rank'] = df1['Hotel_Rank'].astype(str).str.strip().str.lower().replace(dirty_vals,np.nan)

In [124]:
df1["Hotel_Rank_Num"] = df1["Hotel_Rank"].str.extract(r"(\d+(?:\.\d+)?)").astype(float)

In [125]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   num                       740 non-null    int64  
 1   Hotel_ID                  740 non-null    object 
 2   Hotel_Name                740 non-null    object 
 3   Hotel_Rank                267 non-null    object 
 4   Hotel_Address             740 non-null    object 
 5   Total_Score               414 non-null    float64
 6   Location                  413 non-null    float64
 7   Cleanliness               412 non-null    float64
 8   Service                   373 non-null    float64
 9   Facilities                370 non-null    float64
 10  Value_for_money           410 non-null    float64
 11  Comfort_and_room_quality  51 non-null     float64
 12  comments_count            740 non-null    int64  
 13  Hotel_Description         739 non-null    object 
 14  Hotel_Rank

In [126]:
cols_to_fill = ["Total_Score", "Location", "Cleanliness", "Service","Facilities", "Value_for_money", "Comfort_and_room_quality","Hotel_Rank_Num"]

for col in cols_to_fill:
    df1[col] = df1[col].fillna(0)

In [127]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   num                       740 non-null    int64  
 1   Hotel_ID                  740 non-null    object 
 2   Hotel_Name                740 non-null    object 
 3   Hotel_Rank                267 non-null    object 
 4   Hotel_Address             740 non-null    object 
 5   Total_Score               740 non-null    float64
 6   Location                  740 non-null    float64
 7   Cleanliness               740 non-null    float64
 8   Service                   740 non-null    float64
 9   Facilities                740 non-null    float64
 10  Value_for_money           740 non-null    float64
 11  Comfort_and_room_quality  740 non-null    float64
 12  comments_count            740 non-null    int64  
 13  Hotel_Description         739 non-null    object 
 14  Hotel_Rank

In [128]:
df1 = df1.drop(columns=["Hotel_Rank"])

In [129]:
df1["Hotel_Description"] = df1["Hotel_Description"].fillna("-")

In [130]:
df2= pd.read_csv('Data_Agoda_raw/hotel_comments.csv')
df2.head()

,num,Hotel ID,Reviewer ID,Reviewer Name,Nationality,Group Name,Room Type,Stay Details,Score,Score Level,Title,Body,Review Date
0,1,1_1,1_1_1,MARIKO,Nhật Bản,Cặp đôi,Phòng Deluxe 2 Giường đơn Nhìn ra Biển,Đã ở 3 đêm vào Tháng 7 năm 2023,"10,0",Trên cả tuyệt vời,Cao nhất‼︎”,Tôi đã ở cùng chủ nhân trong 4 đêm. Nhân viên ...,Đã nhận xét vào 30 tháng 7 2023
1,2,1_1,1_1_2,Hong,Việt Nam,Đi công tác,Phòng Deluxe 2 Giường đơn Nhìn ra Biển,Đã ở 1 đêm vào Tháng 9 năm 2022,"10,0",Trên cả tuyệt vời,Tháng 8”,Lựa chọn Mường Thanh vì giá cả phù hợp. Đặt On...,Đã nhận xét vào 05 tháng 9 2022
2,3,1_1,1_1_3,Guai,Việt Nam,Cặp đôi,Deluxe Hướng biển giường đôi,Đã ở 1 đêm vào Tháng 6 năm 2024,"9,2",Trên cả tuyệt vời,Du lịch tại Nha Trang”,"Lần này đến với Nha Trang, tôi book phòng tại ...",Đã nhận xét vào 25 tháng 6 2024
3,4,1_1,1_1_4,Nghĩa,Việt Nam,Gia đình có em bé,Deluxe Hướng biển giường đôi,Đã ở 3 đêm vào Tháng 6 năm 2024,"8,8",Tuyệt vời,Du lịch Nha Trang tại Mường Thanh”,Hôm đi đến lúc về thì mọi thứ trong Khách sạn ...,Đã nhận xét vào 02 tháng 7 2024
4,5,1_1,1_1_5,Duc,Việt Nam,Cặp đôi,Deluxe 2 giường Hướng phố,Đã ở 1 đêm vào Tháng 6 năm 2024,"9,2",Trên cả tuyệt vời,Ks tốt !”,Khách sạn có vị trí trung tâm và sát biển. Nhâ...,Đã nhận xét vào 16 tháng 6 2024


In [131]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80314 entries, 0 to 80313
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   num            80314 non-null  int64 
 1   Hotel ID       80314 non-null  object
 2   Reviewer ID    80314 non-null  object
 3   Reviewer Name  80228 non-null  object
 4   Nationality    80314 non-null  object
 5   Group Name     80314 non-null  object
 6   Room Type      80314 non-null  object
 7   Stay Details   80314 non-null  object
 8   Score          80314 non-null  object
 9   Score Level    80314 non-null  object
 10  Title          80314 non-null  object
 11  Body           80272 non-null  object
 12  Review Date    80314 non-null  object
dtypes: int64(1), object(12)
memory usage: 8.0+ MB


In [132]:
df2['Score'] = df2['Score'].astype(str).str.strip().str.lower().replace(dirty_vals, np.nan).str.replace(",", ".", regex=False)

In [133]:
df2["Score"] = df2["Score"].astype(str).str.replace(",", ".").astype(float)

In [134]:
df2.columns = df2.columns.str.strip().str.replace(" ", "_")

In [135]:
df2 = df2.dropna(subset=['Reviewer_Name'])
df2 = df2.dropna(subset=['Body'])

In [136]:
df2 = df2.merge(df1[["Hotel_ID", "Hotel_Name"]], on="Hotel_ID", how="left")
df2.head(2)

,num,Hotel_ID,Reviewer_ID,Reviewer_Name,Nationality,Group_Name,Room_Type,Stay_Details,Score,Score_Level,Title,Body,Review_Date,Hotel_Name
0,1,1_1,1_1_1,MARIKO,Nhật Bản,Cặp đôi,Phòng Deluxe 2 Giường đơn Nhìn ra Biển,Đã ở 3 đêm vào Tháng 7 năm 2023,10.0,Trên cả tuyệt vời,Cao nhất‼︎”,Tôi đã ở cùng chủ nhân trong 4 đêm. Nhân viên ...,Đã nhận xét vào 30 tháng 7 2023,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...
1,2,1_1,1_1_2,Hong,Việt Nam,Đi công tác,Phòng Deluxe 2 Giường đơn Nhìn ra Biển,Đã ở 1 đêm vào Tháng 9 năm 2022,10.0,Trên cả tuyệt vời,Tháng 8”,Lựa chọn Mường Thanh vì giá cả phù hợp. Đặt On...,Đã nhận xét vào 05 tháng 9 2022,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...


In [137]:
df2["Days"] = df2["Stay_Details"].str.extract(r"Đã ở (\d+) đêm").astype(int)

In [138]:
df2["Month_Stay"] = df2["Stay_Details"].str.extract(r"Tháng\s+(\d+)").astype(int)

In [139]:
df2[["day", "month", "year"]] = df2["Review_Date"].str.extract(r"(\d+)\s+tháng\s+(\d+)\s+(\d+)")

In [140]:
df2["Review_Date"] = pd.to_datetime(df2["day"] + "/" + df2["month"] + "/" + df2["year"], format="%d/%m/%Y")

In [141]:
df2.columns

Index(['num', 'Hotel_ID', 'Reviewer_ID', 'Reviewer_Name', 'Nationality',
       'Group_Name', 'Room_Type', 'Stay_Details', 'Score', 'Score_Level',
       'Title', 'Body', 'Review_Date', 'Hotel_Name', 'Days', 'Month_Stay',
       'day', 'month', 'year'],
      dtype='object')

In [142]:
df2 = df2.drop(columns=["num","Reviewer_ID","Stay_Details","day","month","year"])

In [143]:
df2["Mean_Reviewer_Score"] = (df2.groupby("Hotel_ID")["Score"].transform("mean"))

In [144]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80186 entries, 0 to 80185
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Hotel_ID             80186 non-null  object        
 1   Reviewer_Name        80186 non-null  object        
 2   Nationality          80186 non-null  object        
 3   Group_Name           80186 non-null  object        
 4   Room_Type            80186 non-null  object        
 5   Score                80186 non-null  float64       
 6   Score_Level          80186 non-null  object        
 7   Title                80186 non-null  object        
 8   Body                 80186 non-null  object        
 9   Review_Date          80186 non-null  datetime64[ns]
 10  Hotel_Name           69751 non-null  object        
 11  Days                 80186 non-null  int32         
 12  Month_Stay           80186 non-null  int32         
 13  Mean_Reviewer_Score  80186 non-

Những Hotel_ID ở bảng df2 không có tên tại bảng df1 có thể là những hotel_id lỗi không đáng tin tưởng nên drop

In [145]:
df2 = df2.dropna(subset=['Hotel_Name'])

In [146]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69751 entries, 0 to 80185
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Hotel_ID             69751 non-null  object        
 1   Reviewer_Name        69751 non-null  object        
 2   Nationality          69751 non-null  object        
 3   Group_Name           69751 non-null  object        
 4   Room_Type            69751 non-null  object        
 5   Score                69751 non-null  float64       
 6   Score_Level          69751 non-null  object        
 7   Title                69751 non-null  object        
 8   Body                 69751 non-null  object        
 9   Review_Date          69751 non-null  datetime64[ns]
 10  Hotel_Name           69751 non-null  object        
 11  Days                 69751 non-null  int32         
 12  Month_Stay           69751 non-null  int32         
 13  Mean_Reviewer_Score  69751 non-null 

##### Check dup review, comment 

In [147]:
df2['Review_ID_real'] = (
    df2['Hotel_ID'].astype(str) + '_' +
    df2['Reviewer_Name'].astype(str) + '_' +
    df2['Nationality'].astype(str) + '_' +
    df2['Review_Date'].astype(str) 
)

In [148]:
def check_duplicate_column(df, col_name):
    dup_rows = df[df.duplicated(subset=[col_name])]
    print(f"Số dòng trùng theo cột '{col_name}': {len(dup_rows)}")
    return dup_rows

In [149]:
check_duplicate_column(df2, 'Review_ID_real')

Số dòng trùng theo cột 'Review_ID_real': 43845


,Hotel_ID,Reviewer_Name,Nationality,Group_Name,Room_Type,Score,Score_Level,Title,Body,Review_Date,Hotel_Name,Days,Month_Stay,Mean_Reviewer_Score,Review_ID_real
45,1_1,Minh,Việt Nam,Nhóm,Phòng Deluxe 2 Giường đơn Nhìn ra Biển,8.0,Tuyệt vời,Nóng”,Máy lạnh sảnh và phòng ăn hư. Rất nóng.,2024-06-22,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,2,6,9.407407,1_1_Minh_Việt Nam_2024-06-22
46,1_1,Minh,Việt Nam,Cặp đôi,Deluxe Hướng biển giường đôi,8.0,Tuyệt vời,Nóng”,Máy Lạnh sảnh và phòng ăn hư nên rất nóng,2024-06-22,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,2,6,9.407407,1_1_Minh_Việt Nam_2024-06-22
73,1_1,LE,Việt Nam,Cặp đôi,Deluxe 2 giường Hướng phố,10.0,Trên cả tuyệt vời,Tốt”,Sẽ quay lại,2024-04-14,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,2,2,9.407407,1_1_LE_Việt Nam_2024-04-14
74,1_1,LE,Việt Nam,Cặp đôi,Deluxe 2 giường Hướng phố,10.0,Trên cả tuyệt vời,Tốt”,Sẽ quay lại,2024-04-14,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,2,2,9.407407,1_1_LE_Việt Nam_2024-04-14
75,1_1,LE,Việt Nam,Nhóm,Deluxe 2 giường Hướng phố,10.0,Trên cả tuyệt vời,Tốt”,Sẽ quay lại,2024-04-14,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,2,2,9.407407,1_1_LE_Việt Nam_2024-04-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79945,39_7,cheng,Trung Quốc,Gia đình có trẻ em,Suite Gia Đình 2 Phòng Ngủ,8.4,Tuyệt vời,性价比很高的家庭酒店”,装修很新，用的是大金空调，东芝洗衣机，设施硬件非常好，有电梯。价格很公道，早餐也不错。隔壁几...,2019-01-26,Khách sạn & Căn hộ Moonlight Bay (Moonlight Ba...,3,1,9.267742,39_7_cheng_Trung Quốc_2019-01-26
79952,39_7,SEJUNG,Hàn Quốc,Du lịch một mình,Phòng Junior Suite,10.0,Trên cả tuyệt vời,나트랑 최고의 가성비호텔”,"총 3박머무는동안 정말 너무만족스러웠구 혼자온 외국인인 저를위한 세심한배려, 사장님...",2022-08-12,Khách sạn & Căn hộ Moonlight Bay (Moonlight Ba...,1,8,9.267742,39_7_SEJUNG_Hàn Quốc_2022-08-12
79979,39_17,Nino,Thụy Sĩ,Cặp đôi,Phòng Tiêu Chuẩn,10.0,Trên cả tuyệt vời,Trên cả tuyệt vời”,Great,2024-03-03,Gold Coast Hotel Nha Trang,1,2,9.288889,39_17_Nino_Thụy Sĩ_2024-03-03
79980,39_17,Nino,Thụy Sĩ,Cặp đôi,Phòng Tiêu Chuẩn,10.0,Trên cả tuyệt vời,Recommend ”,Clean and new. Good location friendly staff,2024-03-03,Gold Coast Hotel Nha Trang,3,2,9.288889,39_17_Nino_Thụy Sĩ_2024-03-03


In [150]:
df2 = df2.drop_duplicates(subset=['Review_ID_real'], keep='first')

In [151]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25906 entries, 0 to 80185
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Hotel_ID             25906 non-null  object        
 1   Reviewer_Name        25906 non-null  object        
 2   Nationality          25906 non-null  object        
 3   Group_Name           25906 non-null  object        
 4   Room_Type            25906 non-null  object        
 5   Score                25906 non-null  float64       
 6   Score_Level          25906 non-null  object        
 7   Title                25906 non-null  object        
 8   Body                 25906 non-null  object        
 9   Review_Date          25906 non-null  datetime64[ns]
 10  Hotel_Name           25906 non-null  object        
 11  Days                 25906 non-null  int32         
 12  Month_Stay           25906 non-null  int32         
 13  Mean_Reviewer_Score  25906 non-null 

##### Clean body

In [152]:
import re
import pandas as pd
from joblib import Parallel, delayed

In [153]:
# Load dictionary từ file txt
def load_dict(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    return dict(line.strip().split("\t") for line in lines if "\t" in line)

In [154]:
# Load các dict
teencode_dict = load_dict("files/teencode.txt")
english_vn_dict = load_dict("files/english-vnmese.txt")
emoji_dict = load_dict("files/emojicon.txt")

In [155]:
def replace_all_dicts(text, teencode_dict=None, english_vn_dict=None, emoji_dict=None):
    """
    Thay thế theo dict nhưng không dùng regex pattern
    """
    if pd.isna(text):
        return text

    all_dict = {}
    if teencode_dict:
        all_dict.update(teencode_dict)
    if english_vn_dict:
        all_dict.update(english_vn_dict)
    if emoji_dict:
        all_dict.update(emoji_dict)

    # replace lần lượt theo key
    for k, v in all_dict.items():
        if v and isinstance(v, str):   # value không rỗng
            text = text.replace(k, v)

    return text

In [156]:
df2['Body_clean'] = df2['Body'].apply(lambda x: replace_all_dicts(x, teencode_dict, english_vn_dict,emoji_dict))

In [157]:
# Stopwords
stop_words = set()
with open('files/vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    for line in file:
        word = line.strip()
        if word:
            stop_words.add(word)

In [158]:
# Wrongwords
wrong_words = set()
with open('files\wrong-word.txt', 'r', encoding='utf-8') as file:
    for line in file:
        word = line.strip()
        if word:
            wrong_words.add(word)

In [159]:
def clean_text_remove_wrong(text, wrong_words, stop_words):
    if pd.isna(text):
        return text
    
    # 1️⃣ loại bỏ các từ sai
    words = text.split()
    words = [w for w in words if w.lower() not in wrong_words]
    
    # 2️⃣ loại bỏ stopwords
    words = [w for w in words if w.lower() not in stop_words] 
    return ' '.join(words)

In [160]:
df2['Body_clean'] = df2['Body'].apply(lambda x: clean_text_remove_wrong(x, wrong_words, stop_words))

In [161]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25906 entries, 0 to 80185
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Hotel_ID             25906 non-null  object        
 1   Reviewer_Name        25906 non-null  object        
 2   Nationality          25906 non-null  object        
 3   Group_Name           25906 non-null  object        
 4   Room_Type            25906 non-null  object        
 5   Score                25906 non-null  float64       
 6   Score_Level          25906 non-null  object        
 7   Title                25906 non-null  object        
 8   Body                 25906 non-null  object        
 9   Review_Date          25906 non-null  datetime64[ns]
 10  Hotel_Name           25906 non-null  object        
 11  Days                 25906 non-null  int32         
 12  Month_Stay           25906 non-null  int32         
 13  Mean_Reviewer_Score  25906 non-null 

##### Clean body review tiếng nước ngoài

Vì bảng df2 đang có những cmt tiếng anh, tiếng việt và các thứ tiếng khác nên không thể cho insight các từ khóa tích cực, tiêu cực về khách sạn đó.
Vậy nên, bảng df2 sẽ được copy thành bảng df3 và loại bỏ các thứ tiếng khác ngoại trừ các body review tiếng anh và tiếng việt đã được xử lý ở trên

In [162]:
df3 = df2

In [163]:
from langdetect import detect

In [164]:
def is_not_vi_en(text):
    try:
        lang = detect(text)
        return lang not in ['vi', 'en']
    except:
        return False 

In [165]:
df3['non_vi_en'] = df3['Body_clean'].apply(is_not_vi_en)

In [166]:
num_non_vi_en = df3['non_vi_en'].sum()
num_non_vi_en

6767

In [167]:
# Giữ lại những dòng có non_vi_en == False
df3 = df3[df3['non_vi_en'] == False].copy()

In [168]:
# Loại bỏ cmt chỉ có ký tự
def has_letters(text):
    if pd.isna(text):
        return False
    # Kiểm tra có ít nhất 1 ký tự chữ cái (Latin hoặc tiếng Việt có dấu)
    return bool(re.search(r'[A-Za-zÀ-ỹà-ỹ]', text))

In [169]:
# Lọc giữ lại comment có chữ cái
df3 = df3[df3['Body_clean'].apply(has_letters)].copy()

In [170]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17512 entries, 0 to 80185
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Hotel_ID             17512 non-null  object        
 1   Reviewer_Name        17512 non-null  object        
 2   Nationality          17512 non-null  object        
 3   Group_Name           17512 non-null  object        
 4   Room_Type            17512 non-null  object        
 5   Score                17512 non-null  float64       
 6   Score_Level          17512 non-null  object        
 7   Title                17512 non-null  object        
 8   Body                 17512 non-null  object        
 9   Review_Date          17512 non-null  datetime64[ns]
 10  Hotel_Name           17512 non-null  object        
 11  Days                 17512 non-null  int32         
 12  Month_Stay           17512 non-null  int32         
 13  Mean_Reviewer_Score  17512 non-null 

In [171]:
df3.head(2)

,Hotel_ID,Reviewer_Name,Nationality,Group_Name,Room_Type,Score,Score_Level,Title,Body,Review_Date,Hotel_Name,Days,Month_Stay,Mean_Reviewer_Score,Review_ID_real,Body_clean,non_vi_en
0,1_1,MARIKO,Nhật Bản,Cặp đôi,Phòng Deluxe 2 Giường đơn Nhìn ra Biển,10.0,Trên cả tuyệt vời,Cao nhất‼︎”,Tôi đã ở cùng chủ nhân trong 4 đêm. Nhân viên ...,2023-07-30,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,3,7,9.407407,1_1_MARIKO_Nhật Bản_2023-07-30,chủ nhân 4 đêm. Nhân viên thân thiện. Tầm phòn...,False
1,1_1,Hong,Việt Nam,Đi công tác,Phòng Deluxe 2 Giường đơn Nhìn ra Biển,10.0,Trên cả tuyệt vời,Tháng 8”,Lựa chọn Mường Thanh vì giá cả phù hợp. Đặt On...,2022-09-05,Khách sạn Mường Thanh Luxury Nha Trang (Muong ...,1,9,9.407407,1_1_Hong_Việt Nam_2022-09-05,Lựa Mường giá phù hợp. Online ưu đãi. Bữa phú ...,False


In [172]:
hotel_corpus3 = df3.groupby("Hotel_ID")["Body_clean"].apply(lambda x: " ".join(x)).reset_index()

In [173]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17512 entries, 0 to 80185
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Hotel_ID             17512 non-null  object        
 1   Reviewer_Name        17512 non-null  object        
 2   Nationality          17512 non-null  object        
 3   Group_Name           17512 non-null  object        
 4   Room_Type            17512 non-null  object        
 5   Score                17512 non-null  float64       
 6   Score_Level          17512 non-null  object        
 7   Title                17512 non-null  object        
 8   Body                 17512 non-null  object        
 9   Review_Date          17512 non-null  datetime64[ns]
 10  Hotel_Name           17512 non-null  object        
 11  Days                 17512 non-null  int32         
 12  Month_Stay           17512 non-null  int32         
 13  Mean_Reviewer_Score  17512 non-null 

### Collaborative Filtering

##### ALS

In [174]:
import findspark
findspark.init("C:/spark/spark-3.5.1-bin-hadoop3")

In [175]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

In [176]:
spark = SparkSession.builder \
    .appName("Hotel_Recommendation") \
    .getOrCreate()

In [177]:
sdf1 = spark.createDataFrame(df1)
sdf3 = spark.createDataFrame(df3)

In [178]:
sdf1.show(5)

+---+--------+--------------------+--------------------+-----------+--------+-----------+-------+----------+---------------+------------------------+--------------+--------------------+--------------+
|num|Hotel_ID|          Hotel_Name|       Hotel_Address|Total_Score|Location|Cleanliness|Service|Facilities|Value_for_money|Comfort_and_room_quality|comments_count|   Hotel_Description|Hotel_Rank_Num|
+---+--------+--------------------+--------------------+-----------+--------+-----------+-------+----------+---------------+------------------------+--------------+--------------------+--------------+
|  1|     1_1|Khách sạn Mường T...|60 Trần Phú, Lộc ...|        8.8|     9.4|        8.9|    8.9|       8.7|            8.7|                     8.3|          1269|Khách sạn Mường T...|           5.0|
|  2|     1_2|ALPHA BIRD NHA TRANG|51/19/37 Tue Tinh...|        7.7|     7.8|        7.6|    8.1|       7.5|            8.1|                     0.0|           337|ALPHA BIRD NHA TR...|           

In [179]:
sdf3.show(5)

+--------+-------------+-----------+-----------------+--------------------+-----+-----------------+--------------------+--------------------+-------------------+--------------------+----+----------+-------------------+--------------------+--------------------+---------+
|Hotel_ID|Reviewer_Name|Nationality|       Group_Name|           Room_Type|Score|      Score_Level|               Title|                Body|        Review_Date|          Hotel_Name|Days|Month_Stay|Mean_Reviewer_Score|      Review_ID_real|          Body_clean|non_vi_en|
+--------+-------------+-----------+-----------------+--------------------+-----+-----------------+--------------------+--------------------+-------------------+--------------------+----+----------+-------------------+--------------------+--------------------+---------+
|     1_1|       MARIKO|   Nhật Bản|          Cặp đôi|Phòng Deluxe 2 Gi...| 10.0|Trên cả tuyệt vời|         Cao nhất‼︎”|Tôi đã ở cùng chủ...|2023-07-30 00:00:00|Khách sạn Mường T...|   3|

In [180]:
sdf3 = sdf3.join(sdf1.select("Hotel_ID", "Hotel_Address", "Hotel_Description"),on="Hotel_ID",how="left")

In [181]:
sdf3.show(5)

+--------+-------------+-----------+----------------+--------------------+-----+-----------------+--------------------+--------------------+-------------------+----------+----+----------+-------------------+--------------------+--------------------+---------+--------------------+--------------------+
|Hotel_ID|Reviewer_Name|Nationality|      Group_Name|           Room_Type|Score|      Score_Level|               Title|                Body|        Review_Date|Hotel_Name|Days|Month_Stay|Mean_Reviewer_Score|      Review_ID_real|          Body_clean|non_vi_en|       Hotel_Address|   Hotel_Description|
+--------+-------------+-----------+----------------+--------------------+-----+-----------------+--------------------+--------------------+-------------------+----------+----+----------+-------------------+--------------------+--------------------+---------+--------------------+--------------------+
|   10_12|      William|      Na Uy|Du lịch một mình|Phòng Senior Delu...|  9.2|Trên cả tuyệt 

In [182]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col

In [183]:
# Map nationality
user_indexer = StringIndexer(inputCol="Nationality", outputCol="nationality_id")
user_indexer_model = user_indexer.fit(sdf3)
sdf3 = user_indexer_model.transform(sdf3)

In [184]:
# Map hotel
hotel_indexer = StringIndexer(inputCol="Hotel_ID", outputCol="hotel_numeric_id")
hotel_indexer_model = hotel_indexer.fit(sdf3)
sdf3 = hotel_indexer_model.transform(sdf3)

In [185]:
sdf_cf = sdf3.select(
    "nationality_id",     # user
    "hotel_numeric_id",    # item
    "Score",               # rating
    "Body_clean",          # nội dung review
    "Hotel_Description"    # thông tin khách sạn
)

In [186]:
sdf_cf.printSchema()

root
 |-- nationality_id: double (nullable = false)
 |-- hotel_numeric_id: double (nullable = false)
 |-- Score: double (nullable = true)
 |-- Body_clean: string (nullable = true)
 |-- Hotel_Description: string (nullable = true)



In [187]:
sdf_cf = sdf_cf.withColumn("nationality_id", col("nationality_id").cast("int"))
sdf_cf = sdf_cf.withColumn("hotel_numeric_id", col("hotel_numeric_id").cast("int"))

In [188]:
train, test = sdf_cf.randomSplit([0.8, 0.2], seed=42)

In [189]:
print("Train count:", train.count())
print("Test count:", test.count())

Train count: 14063
Test count: 3449


In [190]:
als = ALS(
    userCol="nationality_id",
    itemCol="hotel_numeric_id",
    ratingCol="Score",
    implicitPrefs=False,
    coldStartStrategy="drop",
    rank=20,
    maxIter=10,
    regParam=0.1
)

In [191]:
als_model = als.fit(train)
als_predictions = als_model.transform(test)

In [193]:
als_predictions.select("nationality_id","hotel_numeric_id","Score","prediction").show(10, False)

+--------------+----------------+-----+----------+
|nationality_id|hotel_numeric_id|Score|prediction|
+--------------+----------------+-----+----------+
|0             |148             |8.0  |9.468437  |
|0             |148             |8.8  |9.468437  |
|0             |148             |10.0 |9.468437  |
|0             |148             |10.0 |9.468437  |
|0             |148             |10.0 |9.468437  |
|0             |148             |10.0 |9.468437  |
|4             |148             |10.0 |9.074545  |
|0             |31              |7.2  |8.980207  |
|0             |31              |7.2  |8.980207  |
|0             |31              |7.6  |8.980207  |
+--------------+----------------+-----+----------+
only showing top 10 rows



In [194]:
from pyspark.ml.evaluation import RegressionEvaluator

In [197]:
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="Score",
    predictionCol="prediction"
)

In [198]:
rmse = evaluator.evaluate(als_predictions)
rmse

0.9877484755582976

In [212]:
from pyspark.sql import functions as F

In [210]:
def recommend_hotels_by_ALS(als_model, sdf, nationality_id, top_k=10):
    # Lấy danh sách khách sạn (distinct)
    hotels_df = sdf.select("hotel_numeric_id", "Hotel_ID", "Hotel_Name", "Hotel_Address").distinct()
    
    # Gắn user_id vào để predict
    user_df = hotels_df.withColumn("nationality_id", F.lit(nationality_id))
    
    # Dự đoán bằng ALS
    predictions = als_model.transform(user_df)
    
    # Lọc prediction != null, lấy top-k
    result = predictions.filter(col("prediction").isNotNull()) \
                        .orderBy(col("prediction").desc()) \
                        .limit(top_k)
    
    return result.select("Hotel_ID", "Hotel_Name", "Hotel_Address", "prediction")

In [213]:
als_recs = recommend_hotels_by_ALS(als_model, sdf3, nationality_id=1, top_k=10)
als_recs.show(truncate=False)

+--------+-----------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+----------+
|Hotel_ID|Hotel_Name                                                                                                       |Hotel_Address                                                                                                |prediction|
+--------+-----------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+----------+
|28_24   |InterContinental Residences Nha Trang                                                                            |32 Tran Phu Street, Lộc Thọ, Nha Trang, Việt Nam, 650000                                                     |10.077377 |
|8_27    |Vinpea

##### TF-IDF

In [199]:
from pyspark.sql.functions import concat_ws, coalesce, col, lit

In [200]:
sdf_text = sdf_cf.withColumn(
    "text",
    concat_ws(" ",
        coalesce(col("Body_clean").cast("string"), lit("")),
        coalesce(col("Hotel_Description").cast("string"), lit(""))
    )
)

In [201]:
sdf_text.show(5)

+--------------+----------------+-----+--------------------+--------------------+--------------------+
|nationality_id|hotel_numeric_id|Score|          Body_clean|   Hotel_Description|                text|
+--------------+----------------+-----+--------------------+--------------------+--------------------+
|            15|              35| 10.0|nhân viên tốt. mó...|Khách sạn Balcony...|nhân viên tốt. mó...|
|             0|              35| 10.0|sạn giá phù hợp, ...|Khách sạn Balcony...|sạn giá phù hợp, ...|
|             0|              35| 10.0|10 trải nghiệm sạ...|Khách sạn Balcony...|10 trải nghiệm sạ...|
|             0|              35|  8.0|Phòng tiện mới. n...|Khách sạn Balcony...|Phòng tiện mới. n...|
|             0|              35| 10.0|phòng sạch sẽ, lễ...|Khách sạn Balcony...|phòng sạch sẽ, lễ...|
+--------------+----------------+-----+--------------------+--------------------+--------------------+
only showing top 5 rows



In [202]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=2000)
idf = IDF(inputCol="rawFeatures", outputCol="tfidf_features")

In [203]:
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf])
model = pipeline.fit(sdf_text)
sdf_tfidf = pipeline.fit(sdf_text).transform(sdf_text)

##### Tính Similarity giữa Hotels (Content-based)

In [205]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
import numpy as np

In [206]:
def cosine_sim(a, b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))) if np.linalg.norm(a) and np.linalg.norm(b) else 0.0

cosine_udf = udf(lambda x, y: cosine_sim(x.toArray(), y.toArray()), FloatType())

Kết hợp ALS + TF-IDF

In [207]:
alpha = 0.7

In [208]:
sdf_hybrid = als_predictions.join(
    sdf_tfidf.select("hotel_numeric_id", "tfidf_features"),
    on="hotel_numeric_id",
    how="left"
)

In [209]:
sdf_hybrid.show(5)

+----------------+--------------+-----+--------------------+--------------------+----------+--------------------+
|hotel_numeric_id|nationality_id|Score|          Body_clean|   Hotel_Description|prediction|      tfidf_features|
+----------------+--------------+-----+--------------------+--------------------+----------+--------------------+
|               0|             0|  7.6|Nhân viên nhiệt t...|Maris Hotel Nha T...|  9.687291|(2000,[1,4,21,24,...|
|               0|             0|  7.6|Nhân viên nhiệt t...|Maris Hotel Nha T...|  9.687291|(2000,[1,4,9,21,2...|
|               0|             0|  7.6|Nhân viên nhiệt t...|Maris Hotel Nha T...|  9.687291|(2000,[1,4,21,24,...|
|               0|             0|  7.6|Nhân viên nhiệt t...|Maris Hotel Nha T...|  9.687291|(2000,[1,4,21,24,...|
|               0|             0|  7.6|Nhân viên nhiệt t...|Maris Hotel Nha T...|  9.687291|(2000,[1,4,21,24,...|
+----------------+--------------+-----+--------------------+--------------------+-------